In [1]:
from cforest.forest import CausalForest

In [2]:
# %load simulate.py
"""
This module provides functions for the simulation of data which is used in the
monte carlo simulation.
"""
import numpy as np


def simulate(
    nobs,
    nfeatures,
    coefficients,
    error_var,
    seed,
    propensity_score=None,
    alpha=0.8,
):
    """Simulate data with heterogenous treatment effects.

    Simulate outcomes y (length *nobs*), features X (*nobs* x *nfeatures*) and
    treatment status treatment_status (length *nobs*) using the potential
    outcome framework from Neyman and Rubin.
        We simulate the data by imposing a linear model with two relevant
    features plus a treatment effect. However, we return a design matrix with
    *nfeatures* from which only the two used in the simulation are relevant.
        The model looks the following:
    y(0)_i = coef[0] + X_1i coef[1] +  + X_2i coef[2] + error_i,
    y(1)_i = coef[0] + X_1i coef[1] +  + X_2i coef[2]  + treatment_i + error_i,
    where coef[0], coef[1] and coef[2] denote the intercept and slopes
    respectively, and treatment_i = treatment(X_i) denotes the heterogenous
    treatment effect, which is solely dependent on the location of the
    individual in the feature space of the first two dimensions, i.e. as with
    the linear model it only depends on X_1i and X_2i; at last y(0)_i, y(1)_i
    denote the potential outcomes of individual i.

    Args:
        nobs (int): positive integer denoting the number of observations.
        nfeatures (int): positiv integer denoting the number of features. Must
            be greater than or equal to 2.
        coefficients: Coefficients for the linear model, first value denotes
            the intercept, second and third the slope for the second and third
            feature. Must be convertable to a np.ndarray.
        error_var (float): positive float denoting the error variance.
        seed (int): seed for the random number generator.
        propensity_score (np.array): array containing propensity scores, must
            be of length *nobs*. If None, will be set to 0.5 for each
            individual.
        alpha (float): positive parameter influencing the shape of the
            function. Default is 0.8.

    Returns:
        X (np.array): [nobs x nfeatures] numpy array with simulated features.
        t (np.array): [nobs] boolean numpy array containing treatment
            status of individuals.
        y (np.array): [nobs] numpy array containing "observed" outcomes.

    Raises:
        ValueError, if dimensions mismatch or data type of inputs is incorrect.
    """
    np.random.seed(seed)

    # Assert input values
    if not float.is_integer(float(nobs)):
        raise ValueError("Argument *nobs* is not an integer.")
    if not float.is_integer(float(nfeatures)):
        raise ValueError("Argument *nfeatures* is not an integer.")

    coefficients = np.array(coefficients)
    if nfeatures < 2:
        raise ValueError(
            "Argument *nfeatures* must be greater or equal than" "two"
        )

    if len(coefficients) != 3:
        raise ValueError("Argument *coefficients* needs to be of length 3.")

    # Simulate treatment status
    if propensity_score is None:
        treatment_status = np.random.binomial(1, 0.5, nobs)
    else:
        if len(propensity_score) != nobs:
            raise ValueError(
                "Dimensions of argument *propensity_score* do not"
                "match with *nobs*."
            )
        treatment_status = np.random.binomial(1, propensity_score, nobs)

    error = np.random.normal(0, np.sqrt(error_var), nobs)
    X = np.random.uniform(-15, 15, (nobs, nfeatures))
    y = _compute_outcome(X, coefficients, treatment_status, error, alpha)
    t = np.array(treatment_status, dtype=bool)

    return X, t, y


def _compute_outcome(X, coefficients, treatment_status, error, alpha):
    """Compute observed potential outcome.

    Simulates potential outcomes and returns outcome corresponding to the
    treatment status.

    Args:
        X (np.array): design array containing features.
        coefficients (np.array): coefficient array containing intercept and
            two slope parameter.
        treatment_status (np.array): array containing the treatment status
        error (np.array): array containing the error terms for the linear model
        alpha (float): positive parameter influencing the shape of the function

    Returns:
        y (np.array): the observed potential outcomes.

    >>> import numpy as np
    >>> X = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
    >>> coefficients = np.array([0, 1, -1])
    >>> treatment_status = np.array([True, False, True])
    >>> error = np.array([-0.1, 0, 0.1])
    >>> alpha = 0.2
    >>> _compute_outcome(X, coefficients, treatment_status, error, alpha)
    array([-0.78932461, -1.0, -0.24908566])
    """
    #baseline_model = coefficients[0] + np.dot(X[:, :2], coefficients[1:])
    baseline_model = 0
    y0 = baseline_model + error

    treat_effect = true_treatment_effect(X[:, 0], X[:, 1], alpha)
    y1 = y0 + treat_effect

    y = (1 - treatment_status) * y0 + treatment_status * y1
    return y


def true_treatment_effect(x, y, alpha=0.8, scale=5):
    """Compute individual treatment effects.

    Computes individual treatment effect conditional on features *X* using
    parameter *alpha* to determine the smoothness of the conditional
    treatment function and *scale* to determine the scaling.

    Args:
        X (np.array): array with n rows and 2 columns depicting two features
            for n individuals.
        alpha (float): positive parameter influencing the shape of the
            function. Defaults to 0.8.
        scale (float): positive parameter determining the scaling of the
            function. Defaults to 5. With scale=x the range of the function is
            [0, x].

    Returns:
        result (np.array): array of length n containing individual treatment
            effects.

    >>> import numpy as np
    >>> X = np.array([[0.5, 0.75], [0.25, 1]])
    >>> alpha = 0.2
    >>> _true_treatment_effect(X, alpha)
    array([0.26475042, 0.26442005])
    """
    denominatorx = 1 + np.exp(-alpha * (x - 1 / 3))
    fractionx = 1 / denominatorx

    denominatory = 1 + np.exp(-alpha * (y - 1 / 3))
    fractiony = 1 / denominatory

    result = scale * fractionx * fractiony
    return result


In [3]:
from sklearn.model_selection import train_test_split

In [4]:
simparams = {
    'nobs': 10000,
    'nfeatures': 2,
    'coefficients': [0.5, 0, 0],
    'error_var': 0.1,
    'seed': 1,
    'alpha': 0.975
}


X, t, y = simulate(**simparams)

In [5]:
cf = CausalForest(
    num_trees=50,
    split_ratio=0.5,
    min_leaf=5,
    max_depth=20,
    #use_transformed_outcomes=True,
    num_workers=4,
    seed_counter=1,
)

In [6]:
cf.fit(X, t, y)

In [7]:
XX = np.array([
    [0.5, 0.75], 
    [0.25, 1],
    [0.9, 0.9],
    [0.1, 0.1]
])

In [8]:
cf.predict(XX)

array([0.60700811, 0.92394599, 1.71216153, 0.76622235])